In [26]:
import sys
sys.path.append('../src/')

import numpy as np
import scipy
from scipy.sparse import eye, kron, csr_matrix
from Hamiltonians import Hamiltonians
from tqdm import tqdm
from numba import jit
import cython

from functools import wraps
import time

In [10]:
%load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [31]:
%%cython 

import sys
sys.path.append('../src/')

import numpy as np
import scipy
from scipy.sparse import eye, kron, csr_matrix
from Hamiltonians import Hamiltonians
from tqdm import tqdm
from numba import jit
import cython

from functools import wraps
import time

cpdef psi_dot_psi(list psi1, list psi2):
    cdef double x = 0.
    for i in range(psi1.shape[0]):
        for j in range(psi2.shape[1]):
            x += psi1[i, j] * psi2[i, j]
    return x

In [66]:
%%cython

import sys
sys.path.append('../src/')

import numpy as np
import scipy
from scipy.sparse import eye, kron, csr_matrix
from Hamiltonians import Hamiltonians
from tqdm import tqdm
from numba import jit
import cython

from functools import wraps
import time

from libcpp cimport bool
cimport numpy as np

np.import_array()
DTYPE = np.double

ctypedef np.double_t DTYPE_t


cpdef psi_dot_psi(np.ndarray psi1, np.ndarray psi2):
    cdef double x = 0.
    for i in range(psi1.shape[0]):
        for j in range(psi2.shape[1]):
            x += psi1[i, j] * psi2[i, j]
    return x

cpdef lanczos(int m, double seed, int maxiter, double tol, bint use_seed=False, bint force_maxiter=False):
    cdef list x1 = seed
    cdef list x2 = seed
    cdef list gs = seed
    cdef np.ndarray a = np.zeros(100)
    cdef np.ndarray b = np.zeros(100)
    cdef np.ndarray z = np.zeros((100, 100))
    cdef list lvectors = []
    cdef int control_max = maxiter;
    cdef int e0 = 9999
    # cdef bint force_maxiter = force_maxiter

    # if (maxiter == -1):
    #     force_maxiter = False

    if (control_max == 0):
        gs = 1
        maxiter = 1
        return (e0, gs)

    x1[:, :] = 0
    x2[:, :] = 0
    gs[:, :] = 0
    a[:] = 0.0
    b[:] = 0.0
    if (use_seed):
        x1 = seed
    else:
        for i in range(x1.shape[0]):
            for j in range(x1.shape[1]):
                x1[i, j] = (2 * np.random.random() - 1.)

    #    x1[:,:] = 1
    b[0] = psi_dot_psi(x1, x1)
    b[0] = np.sqrt(b[0])
    x1 = x1 / b[0]
    x2[:] = 0
    b[0] = 1.

    e0 = 9999
    nmax = min(99, maxiter)

    for iter in range(1, nmax + 1):
        eini = e0
        if (b[iter - 1] != 0.):
            aux = x1
            x1 = -b[iter - 1] * x2
            x2 = aux / b[iter - 1]

        aux = m.product(x2)

        x1 = x1 + aux
        a[iter] = psi_dot_psi(x1, x2)
        x1 = x1 - x2 * a[iter]

        b[iter] = psi_dot_psi(x1, x1)
        b[iter] = np.sqrt(b[iter])
        lvectors.append(x2)
        #        print "Iter =",iter,a[iter],b[iter]
        z.resize((iter, iter), refcheck=False)
        z[:, :] = 0
        for i in range(0, iter - 1):
            z[i, i + 1] = b[i + 1]
            z[i + 1, i] = b[i + 1]
            z[i, i] = a[i + 1]
        z[iter - 1, iter - 1] = a[iter]
        d, v = np.linalg.eig(z)

        col = 0
        n = 0
        e0 = 9999
        for e in d:
            if (e < e0):
                e0 = e
                col = n
            n += 1
        e0 = d[col]

        # print("Iter = ",iter," Ener = ",e0)
        if ((force_maxiter and iter >= control_max) or (
                iter >= gs.shape[0] * gs.shape[1] or iter == 99 or abs(b[iter]) < tol) or \
                ((not force_maxiter) and abs(eini - e0) <= tol)):
            # converged
            gs[:, :] = 0.
            for n in range(0, iter):
                gs += v[n, col] * lvectors[n]

            # print("E0 = ", e0)
            maxiter = iter
            return (e0, gs)  # We return with ground states energy

    return (e0, gs)

In file included from /Users/garethjones/.ipython/cython/_cython_magic_2c6a17d27e35dc23b90165e8194b15bd.c:680:
In file included from /Users/garethjones/opt/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/arrayobject.h:4:
In file included from /Users/garethjones/opt/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/ndarrayobject.h:12:
In file included from /Users/garethjones/opt/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/ndarraytypes.h:1944:
/Users/garethjones/opt/anaconda3/lib/python3.9/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: "Using deprecated NumPy API, disable it with "          "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
#warning "Using deprecated NumPy API, disable it with " \
 ^
1 warning generated.


In [42]:
!pip install libcpp

ERROR: Could not find a version that satisfies the requirement libcpp (from versions: none)
ERROR: No matching distribution found for libcpp


In [67]:
class DMRGHamiltonians(Hamiltonians):
    def __init__(self):
        super().__init__()
        self.sz = np.matrix(self.sz.toarray())
        self.splus = np.matrix(self.splus.toarray())

    def heisenberg_interaction(self, H, side='left', size=0, dim=0, splusRL=0, szRL=0):
        if side == 'left':
            H[size] = np.kron(H[size - 1], np.eye(2)) + \
                      np.kron(szRL[size - 1], self.sz) + \
                      0.5 * np.kron(splusRL[size - 1], self.splus.transpose()) + \
                      0.5 * np.kron(splusRL[size - 1].transpose(), self.splus)
        elif side == 'right':
            H[size] = np.kron(np.eye(2), H[size - 1]) + \
                      np.kron(self.sz, szRL[size - 1]) + \
                      0.5 * np.kron(self.splus.transpose(), splusRL[size - 1]) + \
                      0.5 * np.kron(self.splus, splusRL[size - 1].transpose())
        return H[size]

In [72]:
class Position:
    LEFT, RIGHT = range(2)


class DMRG(object):

    def __init__(self, _nsites, _nsweeps, _n_states_to_keep):

        self.nsites = _nsites
        self.n_sweeps = _nsweeps
        self.n_states_to_keep = _n_states_to_keep
        self.nstates = 2
        self.dim_l = 0  # dimension of left block
        self.dim_r = 0  # dimension of right block
        self.left_size = 0  # number of sites in left block
        self.right_size = 0  # number of sites in right block

        self.sz0 = .5 * np.matrix('-1,0;0,1')
        self.splus0 = np.matrix('0,1;0,0')

        self.HL = []  # left block Hamiltonian
        self.HR = []  # right block Hamiltonian
        self.szL = []  # left block Sz
        self.szR = []  # right block Sz
        self.splusL = []  # left block S+
        self.splusR = []  # right block S+

        for i in range(self.nsites):
            self.HL.append(np.zeros(shape=(2, 2)))
            self.HR.append(np.zeros(shape=(2, 2)))
            self.szL.append(self.sz0)
            self.szR.append(self.sz0)
            self.splusL.append(self.splus0)
            self.splusR.append(self.splus0)

        self.psi = np.zeros((2, 2), dtype=np.float)  # ground state wave function
        self.rho = np.zeros((2, 2), dtype=np.float)  # density matrix

        self.energy = 0
        self.error = 0

    def build_block_left(self, it):
        self.left_size = it
        self.dim_l = self.HL[self.left_size-1].shape[0]

        # Enlarge left block
        ham = DMRGHamiltonians()
        # print("Appending to HL index {:d}".format(self.left_size))
        self.HL[self.left_size] = ham.heisenberg_interaction(self.HL, side='left', size=self.left_size,
                                                             dim=self.dim_l, splusRL=self.splusL, szRL=self.szL)

        self.splusL[self.left_size] = np.kron(np.eye(self.dim_l), self.splus0)
        self.szL[self.left_size] = np.kron(np.eye(self.dim_l), self.sz0)

    def build_block_right(self, it):
        self.right_size = it
        self.dim_r = self.HR[self.right_size - 1].shape[0]
        ham = DMRGHamiltonians()
        self.HR[self.right_size] = ham.heisenberg_interaction(self.HR, side='right', size=self.right_size,
                                                             dim=self.dim_r, splusRL=self.splusR, szRL=self.szR)
        self.splusR[self.right_size] = np.kron(self.splus0, np.eye(self.dim_r))
        self.szR[self.right_size] = np.kron(self.sz0, np.eye(self.dim_r))

    def ground_state(self):
        self.dim_l = self.HL[self.left_size].shape[0]
        self.dim_r = self.HR[self.right_size].shape[0]
        self.psi.resize((self.dim_l, self.dim_r))
        maxiter = self.dim_l*self.dim_r
        print("\n")
        print(self)
        print(maxiter)
        (self.energy, self.psi) = lanczos(self, self.psi, maxiter, 1.e-7)
        # (self.energy, self.psi) = np.linalg.eigh(self.psi)

    def density_matrix(self, position):
        if position == Position.LEFT:
            self.rho = np.dot(self.psi, self.psi.transpose())
        else:
            self.rho = np.dot(self.psi.transpose(), self.psi)

    def truncate(self, position, m):
        # diagonalize rho
        rho_eig, rho_evec = scipy.sparse.linalg.eigsh(self.rho)
        self.nstates = m
        rho_evec = np.real(rho_evec)
        rho_eig = np.real(rho_eig)

        # calculate the truncation error for a given number of states m
        # Reorder eigenvectors and trucate
        index = np.argsort(rho_eig)
        # for e in index:
        #     print("RHO EIGENVALUE ", rho_eig[e])
        error = 0.
        if (m < rho_eig.shape[0]):
            for i in range(index.shape[0] - m):
                error += rho_eig[index[i]]
        # print("Truncation error = ", error)

        aux = np.copy(rho_evec)
        if (self.rho.shape[0] > m):
            aux.resize((aux.shape[0], m), refcheck=False)
            n = 0
            for i in range(index.shape[0] - 1, index.shape[0] - 1 - m, -1):
                aux[:, n] = rho_evec[:, index[i]]
                n += 1
        rho_evec = aux

        #        rho_evec = np.eye(self.rho.shape[0])

        # perform transformation:
        U = rho_evec.transpose()
        if (position == Position.LEFT):
            aux2 = np.dot(self.HL[self.left_size], rho_evec)
            self.HL[self.left_size] = np.dot(U, aux2)
            aux2 = np.dot(self.splusL[self.left_size], rho_evec)
            self.splusL[self.left_size] = np.dot(U, aux2)
            aux2 = np.dot(self.szL[self.left_size], rho_evec)
            self.szL[self.left_size] = np.dot(U, aux2)
        else:
            aux2 = np.dot(self.HR[self.right_size], rho_evec)
            self.HR[self.right_size] = np.dot(U, aux2)
            aux2 = np.dot(self.splusR[self.right_size], rho_evec)
            self.splusR[self.right_size] = np.dot(U, aux2)
            aux2 = np.dot(self.szR[self.right_size], rho_evec)
            self.szR[self.right_size] = np.dot(U, aux2)

    def product(self, psi):
        npsi = np.dot(self.HL[self.left_size], psi)
        npsi += np.dot(psi, self.HR[self.right_size].transpose())

        # Sz.Sz
        tmat = np.dot(psi, self.szR[self.right_size].transpose())
        npsi += np.dot(self.szL[self.left_size], tmat)
        # S+.S-
        tmat = np.dot(psi, self.splusR[self.right_size]) * 0.5
        npsi += np.dot(self.splusL[self.left_size], tmat)
        # S-.S+
        tmat = np.dot(psi, self.splusR[self.right_size].transpose()) * 0.5
        npsi += np.dot(self.splusL[self.left_size].transpose(), tmat)

        return npsi

    def _infinite_dmrg(self):
        for i in tqdm(range(1, self.n_sweeps)):
            for iter in range(1, int(self.nsites / 2)):  # do infinite size dmrg for warmup
                # print("WARMUP ITERATION ", iter, S.dim_l, S.dim_r)
                # Create HL and HR by adding the single sites to the two blocks
                self.build_block_left(iter)
                self.build_block_right(iter)

                # find smallest eigenvalue and eigenvector
                self.ground_state()

                # Calculate density matrix
                self.density_matrix(Position.LEFT)

                # Truncate
                self.truncate(Position.LEFT, self.n_states_to_keep)

                # Reflect
                self.density_matrix(Position.RIGHT)
                self.truncate(Position.RIGHT, self.n_states_to_keep)

    def _finite_dmrg(self):
        first_iter = int(self.nsites / 2)
        for i in tqdm(range(1, self.n_sweeps)):
            for sweep in range(1, self.n_sweeps):
                for i in range(first_iter, self.nsites - 3):
                    # print("LEFT-TO-RIGHT ITERATION ", iter, S.dim_l, S.dim_r)
                    # Create HL and HR by adding the single sites to the two blocks
                    self.build_block_left(i)
                    self.build_block_right(self.nsites - i - 2)
                    # find smallest eigenvalue and eigenvector
                    self.ground_state()
                    # Calculate density matrix
                    self.density_matrix(Position.LEFT)
                    # Truncate
                    self.truncate(Position.LEFT, self.n_states_to_keep)
                first_iter = 1
                for i in range(first_iter, self.nsites - 3):
                    # print("RIGHT-TO-LEFT ITERATION ", iter, S.dim_l, S.dim_r)
                    # Create HL and HR by adding the single sites to the two blocks
                    self.build_block_right(i)
                    self.build_block_left(self.nsites - i - 2)
                    # find smallest eigenvalue and eigenvector
                    self.ground_state()
                    # Calculate density matrix
                    self.density_matrix(Position.RIGHT)
                    # Truncate
                    self.truncate(Position.RIGHT, self.n_states_to_keep)

    # @jit(forceobj=True)
    def get_density(self):
        self._infinite_dmrg()
        self._finite_dmrg()
        return self.rho

In [73]:
def run_dmrg_heisenberg():
    nsites = 10
    n_states_to_keep = 10
    n_sweeps = 6
    S = DMRG(nsites, n_sweeps, n_states_to_keep)
    rho = S.get_density()
    print(rho)


if __name__ == '__main__':
    tstrt = time.time()
    run_dmrg_heisenberg()
    tstp = time.time()

    time_taken = tstp-tstrt
    print(time_taken)

/var/folders/lc/0pd94g0j0hx750lshn2s454r0000gn/T/ipykernel_13857/3320328557.py:36: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.psi = np.zeros((2, 2), dtype=np.float)  # ground state wave function
/var/folders/lc/0pd94g0j0hx750lshn2s454r0000gn/T/ipykernel_13857/3320328557.py:37: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  self.rho



16


TypeError: an integer is required

In [ ]:
(int m, double seed, int maxiter, double tol, bint use_seed=False, bint force_maxiter=False)

In [2]:
%%cython

a: cython.int = 0
for i in range(10):
    a += i
print(a)

45


In [3]:
%%cython
cpdef int test(int x):
    cdef int y = 1
    cdef int i
    for i in range(1, x+1):
        y *= i
    return y